In [1]:
import cobra
import cobra.test
import pandas as pd
import numpy as np
import Bio.PDB.Polypeptide as prot
import libsbml
import cobrame

In [79]:
ijo = cobra.io.read_sbml_model('../models/iJO1366.xml')
core = cobra.io.read_sbml_model('../models/ColiCore.xml')
base = cobra.io.read_sbml_model('../models/ColiGS.xml')
pruned = cobra.io.read_sbml_model('../models/ColiPruned.xml')
super_pruned = cobra.io.read_sbml_model('../models/ColiPrunedComp_withoutConsRel.xml')

In [88]:
pruned.objective = pruned.reactions.Bio_T

In [89]:
pruned.optimize()

<Solution 31.65 at 0x109a7b250>

In [90]:
pruned.summary()

IN FLUXES            OUT FLUXES           OBJECTIVES
-------------------  -------------------  ------------
glc_D_e    538       h2o_e     893        Bio_T  31.6
o2_e       392       co2_e     858
nh4_e      341       h_e       827
pi_e        30.4     h2_e      799
so4_e        7.92    ac_e      536
k_e          5.62    Bio        31.6
mg2_e        0.25    5drib_c     0.0141
fe2_e        0.239   hmfurn_c    0.0141
fe3_e        0.225   4hba_c      0.00706
ca2_e        0.15
cl_e         0.15
cobalt2_e    0.0999
cu2_e        0.0999
mn2_e        0.0999
mobd_e       0.0999
zn2_e        0.0999


In [65]:
varner = cobra.io.load_json_model('../models/varner.json')

In [44]:
varner.reactions.query('transcription')

[<Reaction transcriptional_initiation_atoB at 0x109cc5a10>,
 <Reaction transcription_atoB at 0x109cc5a50>,
 <Reaction transcriptional_initiation_Hbd1 at 0x109cc5ad0>,
 <Reaction transcription_Hbd1 at 0x109cc5b10>,
 <Reaction transcriptional_initiation_Crt1 at 0x109cc5b90>,
 <Reaction transcription_Crt1 at 0x109cc5bd0>,
 <Reaction transcriptional_initiation_Ter1 at 0x109cc5c50>,
 <Reaction transcription_Ter1 at 0x109cc5c90>,
 <Reaction transcriptional_initiation_adhE1 at 0x109cc5d10>,
 <Reaction transcription_adhE1 at 0x109cc5d50>]

In [110]:
def extract_txtl_rxns(model):
    aa_metabs = []
    for aa in prot.aa3:
        aa_metabs += model.metabolites.query(aa.lower())
    aa_rxns = []
    for aa_metab in aa_metabs:
        aa_rxns += aa_metab.reactions
    tx_rxns = model.reactions.query('transcription')
    tl_rxns = model.reactions.query('translation')
    txtl_rxns = list(set(aa_rxns).union(tx_rxns).union(tl_rxns))
    return txtl_rxns
txtl_rxns = extract_txtl_rxns(varner)

In [66]:
varner.objective = varner.reactions.but_export

In [74]:
varner.optimize()

<Solution 0.01 at 0x1099aea50>

In [75]:
varner.summary()

IN FLUXES        OUT FLUXES          OBJECTIVES
---------------  ------------------  -------------------
M_glc_D_c  21.7  M_h2o_c   43.4      but_export  0.00532
                 M_co2_c   40
                 M_ac_c    36.8
                 tRNA_c     0.0373
                 M_btol_c   0.00532


In [72]:
def add_enzyme(enz):
    if enz == 'adhE':
        enzs = ['adhE1', 'adhE2']
    else:
        enzs = [enz]
    for enz in enzs:
        rxn = varner.reactions.get_by_id('R_{0}'.format(enz.lower()))
        rxn.add_metabolites(metabolites={varner.metabolites.get_by_id('PROTEIN_{0}'.format(enzs[0])):-1})

In [67]:
varner.reactions.get_by_id('R_atoB')

<Reaction R_atoB at 0x109d8a610>

In [31]:
atob.add_metabolites(metabolites={varner.metabolites.PROTEIN_:-1})

In [69]:
add_enzyme('atoB')

In [73]:
#add_enzyme('Hbd1')
#add_enzyme('Crt1')
#add_enzyme('Ter1')
add_enzyme('adhE')

In [91]:
import cobrame

In [92]:
me = cobrame.MEModel('test')
for met in ijo.metabolites:
    me.add_metabolites(met)
for rxn in ijo.reactions:
    me.add_reaction(rxn)

In [93]:
# "Translational capacity" of organism
me.global_info['kt'] = 4.5  # (in h-1)scott 2010, RNA-to-protein curve fit
me.global_info['r0'] = 0.087  # scott 2010, RNA-to-protein curve fit
me.global_info['k_deg'] = 1.0/5. * 60.0  # 1/5 1/min 60 min/h # h-1

# Molecular mass of RNA component of ribosome
me.global_info['m_rr'] = 1453. # in kDa

# Average molecular mass of an amino acid
me.global_info['m_aa'] = 109. / 1000.  # in kDa

# Proportion of RNA that is rRNA
me.global_info['f_rRNA'] = .86
me.global_info['m_nt'] = 324. / 1000.  # in kDa
me.global_info['f_mRNA'] = .02

# tRNA associated global information
me.global_info['m_tRNA'] = 25000. / 1000.  # in kDA
me.global_info['f_tRNA'] = .12

# Define the types of biomass that will be synthesized in the model
me.add_biomass_constraints_to_model(["protein_biomass", "mRNA_biomass", "tRNA_biomass", "rRNA_biomass",
                                     "ncRNA_biomass", "DNA_biomass", "lipid_biomass", "constituent_biomass",
                                     "prosthetic_group_biomass", "peptidoglycan_biomass"])

In [94]:
names = ['atoB', 'Hbd1', 'Crt1', 'Ter1', 'adhE1']
sequences = []
for name in names:
    with open('../genes/{0}.txt'.format(name.lower()), 'r') as f:
        sequences.append(f.read().strip())

In [ ]:
building.create_transcribed_gene(me, 'b','tRNA', 'ATCG')
building.add_transcription_reaction(me, 'TU_b', {'b'}, sequence)
print(me.reactions.transcription_TU_b.reaction)
me.reactions.transcription_TU_b.update()

In [95]:
from qminospy.me1 import ME_NLP1

me_nlp = ME_NLP1(me, growth_key='mu')
muopt, hs, xopt, cache = me_nlp.bisectmu(precision=1e-6, mumax=1.5)
me.solution.f = me.solution.x_dict['biomass_dilution']

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 0.020267 seconds
Finished substituting S,lb,ub in 0.354135 seconds
Finished makeME_LP in 0.065714 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.75 0.75 1
Finished substituting S,lb,ub in 0.214772 seconds
Finished makeME_LP in 0.053069 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.375 0.375 1
Finished substituting S,lb,ub in 0.212731 seconds
Finished makeME_LP in 0.052668 seconds
Getting MINOS parameters from ME_NLP...
3 0.0 0.0 0.1875 0.1875 1
Finished substituting S,lb,ub in 0.202389 seconds
Finished makeME_LP in 0.057835 seconds
Getting MINOS parameters from ME_NLP...
4 0.0 0.0 0.09375 0.09375 1
Finished substituting S,lb,ub in 0.220595 seconds
Finished makeME_LP in 0.054970 seconds
Getting MINOS parameters from ME_NLP...
5 0.0 0.0 0.046875 0.046875 1
Finished substituting S,lb,ub in 0.205050 seconds
Finished makeME_LP in 0.050162 seconds
Getting MINOS parameters from ME_NLP...

AttributeError: 'NoneType' object has no attribute 'x_dict'

In [96]:
from cobrame.solve.algorithms import binary_search

binary_search(me, min_mu=.1, max_mu=1.5, debug=True, mu_accuracy=1e-6)

LP files will be saved in /var/folders/bw/9j_4j2256t1471ksbdbclntc0000gn/T/tmpc4uh9F
mu       	status	reset	time	iter	obj
0.1000000	-	False	0.92	3755	


/Users/nlarusstone/Documents/MPhil/Research/cf_fba/3rd_Party_Code/cobrame/cobrame/solve/algorithms.py:93 UserWarning: Due to mRNA degradation constraint formulation the model is infeasible at mu = 0. Using mu = .1 instead.


0.1000000	-	False	0.87	3736	


ValueError: 0 needs to be feasible